In [1]:
import sqlite3
import json
import os
import shutil
from pathlib import Path

In [2]:
def get_firefox_cookie_path():
    """Get the path to Firefox's cookies.sqlite file."""
    # if os.name == 'nt':  # Windows
    #     path = Path(os.environ['APPDATA']) / 'Mozilla/Firefox/Profiles'
    # elif os.name == 'posix':  # macOS and Linux
    #     if os.path.exists(str(Path.home() / 'Library')):  # macOS
    path = Path.home() / 'Library/Application Support/Firefox/Profiles'
    #     else:  # Linux
    #         path = Path.home() / '.mozilla/firefox'
    
    # Find the default profile
    profiles = list(path.glob('*.default*'))
    if not profiles:
        raise Exception("No Firefox profile found")
    return profiles[1] / 'cookies.sqlite'

In [3]:
path = Path.home() / 'Library/Application Support/Firefox/Profiles'
path

PosixPath('/Users/pnadel01/Library/Application Support/Firefox/Profiles')

In [4]:
profiles = list(path.glob('*.default*'))
profiles

[PosixPath('/Users/pnadel01/Library/Application Support/Firefox/Profiles/nl97l1mw.default'),
 PosixPath('/Users/pnadel01/Library/Application Support/Firefox/Profiles/3r1wtedk.default-release')]

In [5]:
os.listdir(profiles[1])

['sessionstore-backups',
 'weave',
 'compatibility.ini',
 'settings',
 'favicons.sqlite-wal',
 'extension-store',
 'addons.json',
 '.parentlock',
 'logins.json',
 'search.json.mozlz4',
 'sessionCheckpoints.json',
 'pkcs11.txt',
 'times.json',
 'suggest.sqlite-wal',
 'AlternateServices.bin',
 'extension-preferences.json',
 'addonStartup.json.lz4',
 'suggest.sqlite-shm',
 'suggest.sqlite',
 'crashes',
 'key4.db',
 'webappsstore.sqlite',
 'protections.sqlite',
 'targeting.snapshot.json',
 'places.sqlite-wal',
 'security_state',
 'datareporting',
 'broadcast-listeners.json',
 'cookies.sqlite',
 'domain_to_categories.sqlite-journal',
 'containers.json',
 'formhistory.sqlite',
 'storage',
 'extensions.json',
 'handlers.json',
 'domain_to_categories.sqlite',
 'serviceworker.txt',
 'notificationstore.json',
 'saved-telemetry-pings',
 'bookmarkbackups',
 'content-prefs.sqlite',
 'gmp-gmpopenh264',
 'permissions.sqlite',
 'gmp-widevinecdm',
 'favicons.sqlite',
 'logins-backup.json',
 'webappssto

In [6]:
cookie_path = get_firefox_cookie_path()

In [7]:
temp_path = 'temp_cookies.sqlite'
shutil.copy2(cookie_path, temp_path)

'temp_cookies.sqlite'

In [8]:
domain = "factiva.com"
output_file='cookies.json'

try:
    # Connect to the copied database
    conn = sqlite3.connect(temp_path)
    cursor = conn.cursor()
    
    # Query to get cookies
    query = """
        SELECT name, value, host, path, isSecure, expiry
        FROM moz_cookies
    """
    if domain:
        query += " WHERE host LIKE ?"
        cursor.execute(query, (f"%{domain}%",))
    else:
        cursor.execute(query)
        
    # Convert cookies to list of dictionaries
    cookie_list = []
    for row in cursor.fetchall():
        cookie_dict = {
            'name': row[0],
            'value': row[1],
            'domain': row[2] if row[2].startswith('.') else '.' + row[2],
            'path': row[3],
            'secure': bool(row[4]),
            'expiry': row[5]
        }
        cookie_list.append(cookie_dict)
        
    # Save to file
    with open(output_file, 'w') as f:
        json.dump(cookie_list, f, indent=2)
        
    print(f"Exported {len(cookie_list)} cookies to {output_file}")
    
finally:
    conn.close()
    os.remove(temp_path)

Exported 8 cookies to cookies.json
